# 前置作業

## 安裝套件

In [ ]:
!pip install requests

In [ ]:
!pip install selenium

## import 套件

In [1]:
# coding=utf-8
import requests
import json

from requests import exceptions
import time
import os
import re
import math
import logging
import ast
from pathlib import Path
from IPython.display import clear_output

import sys

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

## 帳號密碼宣告

In [2]:
account = "chickenrun4@yahoo.com.tw"
password = "fcuiecs123"

## 資料夾名稱、Logging紀錄

In [3]:
import logging
import datetime
import pickle

save_file = "20210712"
# 開路徑
Path(f"./{save_file}").mkdir(parents=True, exist_ok=True)

# logging文件設定
str_time = datetime.datetime.now().strftime("%Y-%m-%d_%H_%M_%S.log")

logging.basicConfig(
    filename=f"./{save_file}/LOG {str_time}",
    encoding="utf-8",
    level=logging.INFO,
    # 其中%(levelname)-5s這部分。-5s的意思是向左對齊，總長度為5的字串。
    format= "[%(levelname)-5s %(asctime)s LINE:%(lineno)d] %(message)s",
    datefmt="%Y%m%d_%H:%M:%S",
)

# logging.info('一些資訊')
# logging.error("出了一些BUG")
# try:
#     1/0
# except ZeroDivisionError:
#     logging.exception("發生BUG啦")

## 一題目基本屬性class宣告

In [4]:
class question_info:
    def __init__(self, ID, slug, front_id, has_solution):
        self.ID = ID
        self.slug = slug
        self.front_id = front_id
        
        self.has_solution = has_solution
        self.solution_id = None
#         self.discuss_comment_count = None

    def get_questionData(self):
        json_file = questionData(self.slug)
        with open(f"./{save_file}/{self.ID} {self.slug}/questionData.json", 'w', encoding='utf-8') as f:
            json.dump(json_file, f, ensure_ascii=False, indent=4)
            
    def get_questionTags(self):
        json_file = questionTags(self.slug)
        with open(f"./{save_file}/{self.ID} {self.slug}/questionTags.json", 'w', encoding='utf-8') as f:
            json.dump(json_file, f, ensure_ascii=False, indent=4)
                
    def get_questionTopicCount(self):
        json_file = questionTopicCount(self.ID, self.slug)
        with open(f"./{save_file}/{self.ID} {self.slug}/questionTopicCount.json", 'w', encoding='utf-8') as f:
            json.dump(json_file, f, ensure_ascii=False, indent=4)
                
    def get_discussQuestionTopicTags(self):
        json_file = discussQuestionTopicTags(self.ID, self.slug)
        with open(f"./{save_file}/{self.ID} {self.slug}/discussQuestionTopicTags.json", 'w', encoding='utf-8') as f:
            json.dump(json_file, f, ensure_ascii=False, indent=4)
    
    
    # 以下皆需要有solution才會提供    
    def get_QuestionNote(self):
        if not self.has_solution:
            logging.warning(f"題目: {self.slug}沒有提供solution，請物呼叫抓取solution相關的涵式。")
            return
        json_file = QuestionNote(self.slug)
        with open(f"./{save_file}/{self.ID} {self.slug}/QuestionNote.json", 'w', encoding='utf-8') as f:
            json.dump(json_file, f, ensure_ascii=False, indent=4)
        """❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄ Fetching Topic ID ❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄"""
        try:
            string = json_file['data']['question']['article']
            regex = re.compile(r'"topicId": (\d+)')
            match = regex.search(string)
            self.solution_id = match.group(1)
        except AttributeError as e:
            logging.exception(f"在獲取topicId時，其QuestionNote_OP值為空")
            print("發生錯誤!請查閱LOG檔")
        except exceptions as e:
            logging.exception(f"在獲取topicId時發生未知錯誤\n")
            print("發生錯誤!請查閱LOG檔")
        """❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄ Fetching Topic ID ❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄"""
        
    def get_DiscussTopic(self):
        if not self.has_solution:
            logging.warning(f"題目: {self.slug}沒有提供solution，請物呼叫抓取solution相關的涵式。")
            return
        elif self.solution_id == None:
            logging.warning(f"題目: {self.slug}還沒有抓到topic id，請先使用get_QuestionNote抓到solution的資料。")
            return
        json_file = DiscussTopic(self.slug, self.solution_id)
        with open(f"./{save_file}/{self.ID} {self.slug}/DiscussTopic.json", 'w', encoding='utf-8') as f:
            json.dump(json_file, f, ensure_ascii=False, indent=4)

# 開啟瀏覽器cookie。需要手動通過recaptch!!! 會員登入cookie轉移。

In [5]:
session = requests.Session()
# 網頁cookie之類的
user_agent = r'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'


CHROMEDRIVER_PATH = "./chromedriver.exe"
url = "https://leetcode.com/accounts/login/"

options = Options()
# options.headless = True
driver = webdriver.Chrome(CHROMEDRIVER_PATH, options=options)
driver.implicitly_wait(10) # seconds
driver.get(url)

xpath = '//*[@id="id_login"]'
account_blank = driver.find_element_by_xpath(xpath)
xpath = '//*[@id="id_password"]'
password_blank = driver.find_element_by_xpath(xpath)
xpath = '//*[@id="recaptcha-anchor"]'

account_blank.send_keys(account)
password_blank.send_keys(password)

In [6]:
driver.get_cookies()

[{'domain': '.leetcode.com',
  'expiry': 1941532422,
  'httpOnly': False,
  'name': '87b5a3c3f1a55520_gr_cs1',
  'path': '/',
  'secure': False,
  'value': 'gogochicken'},
 {'domain': '.leetcode.com',
  'expiry': 1626174222,
  'httpOnly': False,
  'name': '87b5a3c3f1a55520_gr_last_sent_sid_with_cs1',
  'path': '/',
  'secure': False,
  'value': 'bd0cf39e-c50c-42f9-9001-f9c8ac019772'},
 {'domain': '.leetcode.com',
  'httpOnly': True,
  'name': 'messages',
  'path': '/',
  'sameSite': 'Lax',
  'secure': True,
  'value': '"8b92ab4a51023ad07481830e3ccdc38c90f52fb3$[[\\"__json_message\\"\\0540\\05425\\054\\"Successfully signed in as gogochicken.\\"]]"'},
 {'domain': '.leetcode.com',
  'expiry': 1941532422,
  'httpOnly': False,
  'name': 'gr_user_id',
  'path': '/',
  'secure': False,
  'value': '5cd68081-80d3-475a-bf23-fb0122d4c1a5'},
 {'domain': 'leetcode.com',
  'expiry': 1657622022,
  'httpOnly': False,
  'name': 'csrftoken',
  'path': '/',
  'sameSite': 'Lax',
  'secure': True,
  'value

In [7]:
cookies = driver.get_cookies()
for cookie in cookies:
    session.cookies.set(cookie['name'], cookie['value'])
driver.close()

# Main 執行區塊

## 拿題目列表

In [16]:
problemset_lst = get_problemset()

1932
skip: 0, i: 0
skip: 50, i: 1
skip: 100, i: 2
skip: 150, i: 3
skip: 200, i: 4
skip: 250, i: 5
skip: 300, i: 6
skip: 350, i: 7
skip: 400, i: 8
skip: 450, i: 9
skip: 500, i: 10
skip: 550, i: 11
skip: 600, i: 12
skip: 650, i: 13
skip: 700, i: 14
skip: 750, i: 15
skip: 800, i: 16
skip: 850, i: 17
skip: 900, i: 18
skip: 950, i: 19
skip: 1000, i: 20
skip: 1050, i: 21
skip: 1100, i: 22
skip: 1150, i: 23
skip: 1200, i: 24
skip: 1250, i: 25
skip: 1300, i: 26
skip: 1350, i: 27
skip: 1400, i: 28
skip: 1450, i: 29
skip: 1500, i: 30
skip: 1550, i: 31
skip: 1600, i: 32
skip: 1650, i: 33
skip: 1700, i: 34
skip: 1750, i: 35
skip: 1800, i: 36
skip: 1850, i: 37
skip: 1900, i: 38


In [17]:
QList = []
for problem in problemset_lst:
    ID = problem['questionId']
    slug = problem['titleSlug']
    front_id = problem['frontendQuestionId']
    has_solution = problem['hasSolution']
    QInfo = question_info(ID, slug, front_id, has_solution)
    QList.append(QInfo)

In [ ]:
len(problemset_lst)

In [ ]:
problemset_lst[0]

In [ ]:
print(problemset_lst[0]['questionId'])
print(problemset_lst[0]['titleSlug'])

In [ ]:
QList[1931].get_info()

In [ ]:
QList[1931].ID

In [ ]:
# 或讀取檔案
with open(f"./{save_file}/problemset_lst.pickle", "rb") as f:
    problemset_lst = pickle.load(f)

## 爬蟲Main區塊

In [29]:
os.path.exists(f'{save_file}/{QInfo.ID} {QInfo.slug}/file_list[0]')

False

In [26]:
QInfo.ID, QInfo.front_id, QInfo.slug

('432', '432', 'all-oone-data-structure')

In [39]:
Path(f"./{save_file}/{QInfo.ID} {QInfo.slug}").mkdir(parents=True, exist_ok=True)
if not (os.path.exists(f'{save_file}/{QInfo.ID} {QInfo.slug}/{file_list[0]}')):
    print("OK")

In [41]:
file_list = [
    "questionData.json",
    "questionTags.json",
    "questionTopicCount.json",
    "discussQuestionTopicTags.json",
    "QuestionNote.json",
    "DiscussTopic.json",
]

for QInfo in QList:
    logging.info(f"題目名稱:{QInfo.slug}；前端ID:{QInfo.front_id}；實際ID:{QInfo.ID}；有無解答:{QInfo.has_solution}")
    logging.info(f"https://leetcode.com/problems/{QInfo.slug}")

    Path(f"./{save_file}/{QInfo.ID} {QInfo.slug}").mkdir(parents=True, exist_ok=True)
    
    if not (os.path.exists(f'{save_file}/{QInfo.ID} {QInfo.slug}/{file_list[0]}')):
        QInfo.get_questionData()
    if not (os.path.exists(f'{save_file}/{QInfo.ID} {QInfo.slug}/{file_list[1]}')):
        QInfo.get_questionTags()
    if not (os.path.exists(f'{save_file}/{QInfo.ID} {QInfo.slug}/{file_list[2]}')):
        QInfo.get_questionTopicCount()
    if not (os.path.exists(f'{save_file}/{QInfo.ID} {QInfo.slug}/{file_list[3]}')):
        QInfo.get_discussQuestionTopicTags()
        
    if (QInfo.has_solution):    
        if not (os.path.exists(f'{save_file}/{QInfo.ID} {QInfo.slug}/{file_list[4]}')):
            QInfo.get_QuestionNote()
        if not (os.path.exists(f'{save_file}/{QInfo.ID} {QInfo.slug}/{file_list[5]}')):
            QInfo.get_DiscussTopic()

In [ ]:
QInfo = QList[1800]

logging.info(f"題目名稱:{QInfo.slug}；前端ID:{QInfo.front_id}；實際ID:{QInfo.ID}；有無解答:{QInfo.has_solution}")
logging.info(f"https://leetcode.com/problems/{QInfo.slug}")

Path(f"./{save_file}/{QInfo.ID} {QInfo.slug}").mkdir(parents=True, exist_ok=True)

QInfo.get_questionData()
QInfo.get_questionTags()
QInfo.get_questionTopicCount()
QInfo.get_discussQuestionTopicTags()

if (QInfo.has_solution):
    QInfo.get_QuestionNote()
    QInfo.get_DiscussTopic()

In [ ]:
QInfo = QList[0]

In [ ]:
QInfo.has_solution

In [ ]:
json.JSONDecodeError

In [ ]:
res = discussQuestionTopicTags(1261, 'find-elements-in-a-contaminated-binary-tree')

In [ ]:
res['data']['discussQuestionTopicTags']

In [ ]:
with open(f"./{save_file}/Filtered_QList.txt", "r") as f:
    QList = [ast.literal_eval(line) for line in f]
QList

# 爬蟲Function

## POST方法宣告

In [8]:
def POST(url, json_data, headers, info = ""):
    try:
        logging.info(f"POST CALL...{info}")
        resp = session.post(url, data = json_data, headers = headers, timeout = 120)
        time.sleep(1)
        return resp.json()
    except requests.exceptions.Timeout as e:
        logging.exception(f"POST讀取超時。階段: {info}。")
        print("發生錯誤!請查閱LOG檔")
        return False
    except json.decoder.JSONDecodeError as e:
        logging.exception(f"JSON解析時發生錯誤。階段: {info}\nRESP:{resp}。")
        print("發生錯誤!請查閱LOG檔")
        return False
    except Exception as e:
        logging.exception(f"發生未知錯誤。階段: {info}。")
        print("發生錯誤!請查閱LOG檔")
        return False

## problemsetQuestionList: 拿到題目列表
Input: None, Output: 題目列表JSON

In [9]:
# get_problemset 拿到所有題目基本資訊
def get_problemset():
    url = "https://leetcode.com/graphql"
    
    query_string = """
query problemsetQuestionList(
  $categorySlug: String
  $limit: Int
  $skip: Int
  $filters: QuestionListFilterInput
) {
  problemsetQuestionList: questionList(
    categorySlug: $categorySlug
    limit: $limit
    skip: $skip
    filters: $filters
  ) {
    total: totalNum
    questions: data {
      acRate
      difficulty
      freqBar
      questionId
      frontendQuestionId: questionFrontendId
      isFavor
      paidOnly: isPaidOnly
      status
      title
      titleSlug
      topicTags {
        name
        id
        slug
      }
      hasSolution
      hasVideoSolution
    }
  }
}
    """
    
    skip = 0
    limit = 50
    variables ={
      "categorySlug": "",
      "skip": skip,
      "limit": limit,
      "filters": {}
    }
    
    params = {'variables': variables,
        'query': query_string}

    json_data = json.dumps(params).encode('utf8')
    headers = {'User-Agent': user_agent, 'Connection': 
        'keep-alive', 'Content-Type': 'application/json',
        'Referer': 'https://leetcode.com/problemset/all/'}
    
    # 先拿到題目數量
    resp = POST(url, json_data, headers, info = "先拿到總題目數量")
    if not resp: return
    total_num = resp['data']['problemsetQuestionList']['total']
    logging.info(f"Get number of problem sets: {total_num}")
    print(total_num)

    
    """ ❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄ 根據所有題目數量，開始抓取所有題目基本資料 ❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄ """
    problemset_lst = []
    for i in range(int(total_num/limit)+1):
        skip = i*limit
        print(f"skip: {skip}, i: {i}")
        
        
        # Variable settings
        variables ={
          "categorySlug": "",
          "skip": skip,
          "limit": limit,
          "filters": {}
        }
        params = {'variables': variables,
            'query': query_string}

        json_data = json.dumps(params).encode('utf8')
        headers = {'User-Agent': user_agent, 'Connection': 
            'keep-alive', 'Content-Type': 'application/json',
            'Referer': 'https://leetcode.com/problemset/all/'}
        
        # POST拿回資料        
        json_data = POST(url, json_data, headers, info = "拿到所有題目基本資訊 problemsetQuestionList")
        if not json_data: return
        for data in json_data['data']['problemsetQuestionList']['questions']:
            problemset_lst.append(data)
    """ ❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄ 根據所有題目數量，開始抓取所有題目基本資料 ❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄ """        
    
    
    # 使用pickle存 problemset_lst
    with open(f"./{save_file}/problemset_lst.pickle", 'wb') as f:
        pickle.dump(problemset_lst, f)
    return problemset_lst

## questionData: 題目基本資訊
Input: ID, slug, Output: questionData_OP JSON

In [10]:
# questionData: 題目基本資訊
def questionData(slug):
    url = "https://leetcode.com/graphql"
    
    operationName = "questionData"
    query_string = """
query questionData($titleSlug: String!) {
  question(titleSlug: $titleSlug) {
    questionId
    questionFrontendId
    boundTopicId
    title
    titleSlug
    content
    translatedTitle
    translatedContent
    isPaidOnly
    difficulty
    likes
    dislikes
    isLiked
    similarQuestions
    exampleTestcases
    contributors {
      username
      profileUrl
      avatarUrl
      __typename
    }
    topicTags {
      name
      slug
      translatedName
      __typename
    }
    companyTagStats
    codeSnippets {
      lang
      langSlug
      code
      __typename
    }
    stats
    hints
    solution {
      id
      canSeeDetail
      paidOnly
      hasVideoSolution
      paidOnlyVideo
      __typename
    }
    status
    sampleTestCase
    metaData
    judgerAvailable
    judgeType
    mysqlSchemas
    enableRunCode
    enableTestMode
    enableDebugger
    envInfo
    libraryUrl
    adminUrl
    __typename
  }
}
    """
    
    variables = {"titleSlug": slug}
    
    """❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄ Every operator is same ❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄"""
    params = {'operationName': operationName,
        'variables': variables,
        'query': query_string}

    json_data = json.dumps(params).encode('utf8')
    headers = {'User-Agent': user_agent, 'Connection': 
        'keep-alive', 'Content-Type': 'application/json',
        'Referer': 'https://leetcode.com/problems/' + slug}
    """❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄ Every operator is same ❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄"""
    
    # 傳回Res資料，需要再.json解析
    resp = POST(url, json_data, headers, info = "抓取題目基本資訊 questionData")
    return resp

## questionTags: 列出一系列的題目類型與面試公司Tag，每個題目都一致

In [11]:
def questionTags(slug):
    url = "https://leetcode.com/graphql"
    
    operationName = "questionTags"
    query_string = """
query questionTags($skipCompanyTags: Boolean = false) {
  questionTopicTags {
    edges {
      node {
        name
        translatedName
        slug
        __typename
      }
      __typename
    }
    __typename
  }
  questionCompanyTags @skip(if: $skipCompanyTags) {
    edges {
      node {
        name
        translatedName
        slug
        __typename
      }
      __typename
    }
    __typename
  }
}
    """
    
    # 付費會員為false
    variables = {"skipCompanyTags": False}
    
    params = {'operationName': operationName,
        'variables': variables,
        'query': query_string}

    json_data = json.dumps(params).encode('utf8')
    headers = {'User-Agent': user_agent, 'Connection': 
        'keep-alive', 'Content-Type': 'application/json',
        'Referer': 'https://leetcode.com/problems/' + slug}

    resp = POST(url, json_data, headers, info = "抓取題目Tag questionTags")
    return resp

## questionTopicCount: 總題目討論串數量

In [12]:
def questionTopicCount(ID, slug):
    url = "https://leetcode.com/graphql"
    
    operationName = "questionTopicCount"
    query_string = """
query questionTopicCount($questionId: String!) {
  questionTopicsList(questionId: $questionId) {
    totalNum
    __typename
  }
}
    """
    
    variables = {"questionId": ID}
    
    params = {'operationName': operationName,
        'variables': variables,
        'query': query_string}

    json_data = json.dumps(params).encode('utf8')
    headers = {'User-Agent': user_agent, 'Connection': 
        'keep-alive', 'Content-Type': 'application/json',
        'Referer': 'https://leetcode.com/problems/' + slug}
    
    resp = POST(url, json_data, headers, info = "抓取總討論串數量 questionTopicCount")
    return resp

## discussQuestionTopicTags: 拿到討論區Tags 
Input: ID,slug, Output: Tags JSON (傳回Res資料，需要再.json解析)

In [13]:
def discussQuestionTopicTags(ID, slug):
    url = "https://leetcode.com/graphql"
    operationName = "discussQuestionTopicTags"
    
    # Same query when fetching different questions
    query = """
    query discussQuestionTopicTags(
      $tagType: String
      $questionId: String!
      $selectedTags: [String!]
    ) {
      discussQuestionTopicTags(
        tagType: $tagType
        questionId: $questionId
        selectedTags: $selectedTags
      ) {
        ...TopicTag
        __typename
      }
    }

    fragment TopicTag on DiscussTopicTagNode {
      id
      name
      slug
      numTopics
      __typename
    }
    """
    
    # Different from questions to questions
    variables = {
      "selectedTags": [],
      "questionId": ID
    }
    
    
    """❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄ Every operator is same ❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄"""
    
    params = {'operationName': operationName,
        'variables': variables,
        'query': query}

    json_data = json.dumps(params).encode('utf8')
    headers = {'User-Agent': user_agent, 'Connection': 
        'keep-alive', 'Content-Type': 'application/json',
        'Referer': 'https://leetcode.com/problems/' + slug}
    
    """❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄ Every operator is same ❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄❄"""
    
    # 傳回Res資料，需要再.json解析
    resp = POST(url, json_data, headers, info = "題目討論區Tag蒐集 discussQuestionTopicTags")
    return resp

## 以下Function題目有要Solution才叫的出來

### QuestionNote: solution相關、內涵 topicId 這題標題的ID，和題目ID不同

In [14]:
def QuestionNote(slug):
    url = "https://leetcode.com/graphql"
    
    operationName = "QuestionNote"
    query_string = """
query QuestionNote($titleSlug: String!) {
  question(titleSlug: $titleSlug) {
    questionId
    article
    solution {
      id
      content
      contentTypeId
      canSeeDetail
      paidOnly
      hasVideoSolution
      paidOnlyVideo
      rating {
        id
        count
        average
        userRating {
          score
          __typename
        }
        __typename
      }
      __typename
    }
    __typename
  }
}
    """
    
    variables = {"titleSlug": slug}
    
    params = {'operationName': operationName,
        'variables': variables,
        'query': query_string}

    json_data = json.dumps(params).encode('utf8')
    headers = {'User-Agent': user_agent, 'Connection': 
        'keep-alive', 'Content-Type': 'application/json',
        'Referer': 'https://leetcode.com/problems/' + slug}

    resp = POST(url, json_data, headers, info = "題目solution相關資訊 QuestionNote")
    return resp

### DiscussTopic 內含 topLevelCommentCount 討論串數量，需要topicId

In [15]:
def DiscussTopic(slug, topicId):
    url = "https://leetcode.com/graphql"
    
    operationName = "DiscussTopic"
    query_string = """
query DiscussTopic($topicId: Int!) {
  topic(id: $topicId) {
    id
    viewCount
    topLevelCommentCount
    subscribed
    title
    pinned
    tags
    hideFromTrending
    post {
      ...DiscussPost
      __typename
    }
    __typename
  }
}

fragment DiscussPost on PostNode {
  id
  voteCount
  voteStatus
  content
  updationDate
  creationDate
  status
  isHidden
  coinRewards {
    ...CoinReward
    __typename
  }
  author {
    isDiscussAdmin
    isDiscussStaff
    username
    nameColor
    activeBadge {
      displayName
      icon
      __typename
    }
    profile {
      userAvatar
      reputation
      __typename
    }
    isActive
    __typename
  }
  authorIsModerator
  isOwnPost
  __typename
}

fragment CoinReward on ScoreNode {
  id
  score
  description
  date
  __typename
}
    """
    
    variables = {"topicId": topicId}
    
    params = {'operationName': operationName,
        'variables': variables,
        'query': query_string}

    json_data = json.dumps(params).encode('utf8')
    headers = {'User-Agent': user_agent, 'Connection': 
        'keep-alive', 'Content-Type': 'application/json',
        'Referer': 'https://leetcode.com/problems/' + slug}

    resp = POST(url, json_data, headers, info = "Solution下的討論串 DiscussTopic")
    return resp